<a href="https://colab.research.google.com/github/kocurvik/edu/blob/master/PNNPPV/notebooky/cv08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8. cvičenie - Vizualizácia

Na dnešnom cvičení si prejdeme metódy vizualizácie dát. Najprv si zobrazíme dôležité časti siete. Potom si zobrazíme ako fungujú filtre.

## Vizualizácia častí obrazu dôležitých pre sieť

Teraz si skúsime overiť ktoré časti v obraze sú pre klasifikáciu dôležité. To spravíme tak, že si vezmeme testovací obrázok zistíme ako sieť klasifikuje. Potom do obrázka budeme vkladať na rôzne miesta čierny štvorec. Potom zistíme ako veľmi sa zmenila predpoveď modelu. Z týchto zmien potom spravíme heatmapu.

In [0]:
import keras
import cv2
import numpy as np
from keras.applications.resnet50 import preprocess_input, decode_predictions
import matplotlib.pyplot as plt

Toto spravíme s predtrénovaným modelom.

In [0]:
resnet = keras.applications.resnet.ResNet50()

Načítame si testovací obrázok a zmeníme jeho rozmery.

In [0]:
test_imgs = []
!wget https://pixnio.com/free-images/2017/06/08/2017-06-08-13-53-59-900x576.jpg
test_imgs.append(cv2.resize(cv2.imread('2017-06-08-13-53-59-900x576.jpg'),(224,224)))
!wget https://storage.needpix.com/rsynced_images/diver-1881751_1280.jpg
test_imgs.append(cv2.resize(cv2.imread('diver-1881751_1280.jpg'),(224,224)))
!wget https://cdn.pixabay.com/photo/2017/09/22/23/24/white-stork-2777489_960_720.jpg
test_imgs.append(cv2.resize(cv2.imread('white-stork-2777489_960_720.jpg'),(224,224)))
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Croatia_police_van_%2804%29.JPG/800px-Croatia_police_van_%2804%29.JPG
test_imgs.append(cv2.resize(cv2.imread('800px-Croatia_police_van_(04).JPG'),(224,224)))


Pred vstupom do siete ešte aplikujeme predspracovanie, aby to korešpondovalo s natrénovanou sieťou. **Treba si taktiež dať pozor na to či je formát v BGR ako to má OpenCV, alebo RGB!**

In [0]:
pred = resnet.predict(preprocess_input(test_imgs[0])[None, :, :, :])
print(np.argmax(pred[0]))
print(decode_predictions(pred)[0])

Túto funkciu napíšte tak aby vrátila heatmapu veľkosti $rectangle\_num \times rectangle\_num$. Jednotlivé elementy heatmapy budú reprezentovať to ako veľmi sa zmení predikcia pre pôvodnú triedu obrázku ak na korešpondujúce miesto v obrázku pridáme čierny štvorec veľkosti $rectangle\_size \times rectangle\_size$.

Pri implementácii skúste využiť to, že metóda model.predict dokáže spracovať viacero obrázkov naraz ak ich poukladáte "na seba" podobne ako pri generovaní batchu.

In [0]:
def generate_heatmap(img, model, rectangle_num, rectangle_size):
  heatmap = np.zeros([rectangle_num, rectangle_num])
  return heatmap

Tento kód by mal zobraziť heatmapy pre vybrané obrázky.

In [0]:
for test_img in test_imgs:
  hmap = generate_heatmap(test_img, resnet, 20, 9)
  plt.imshow(hmap,cmap='gray')
  plt.show()
  plt.imshow(test_img)
  plt.show()

## DeepDream

V tejto časti si predvedieme základný princíp metódy DeepDream, ktorá núti sieť "halucinovať" silnejšie príznaky. Tu si ukážeme len základný algoritmus. V skutočnosti sa používa napríklad vo viacerych škálach a medzi jednotlivými sa obrázok ešte posúva. Plnú verziu nájdete v [notebooku pre DOD 2019](https://github.com/kocurvik/edu/blob/master/DOD/DOD_2019_DeepDream.ipynb), alebo v [originálnej verzii](https://research.google.com/seedbank/seed/deepdream).

Princíp DeepDreamu je jednoduchý. Vyberieme časť siete (stačí aj jeden kanál nejakej feature mapy) a jeho hodnotu (jej štvorec) si označíme ako cenovú funkciu. Potom spočítame gradient obrazu voči tejto cene. To samozrejme framework dokáže jednoducho. Vďaka gradientu potom môžme modifikovať obraz tak aby sa naša cenová funkcia zväčšovala. Tým dostaneme obraz, ktorý aktivuje danú časť siete čo najviac.

Na to aby to fungovalo budeme potrebovať znova volať backend. Užitočná bude funkcia K.gradients. Takisto budeme potrebovať vedieť nájsť časti siete ktoré nás zaujímajú. To docielime pomocou model.layers a vstupný obraz dostaneme pomocou model.input.



In [0]:
import keras.backend as K

Tu je vytvorená funkcia inverzná k preprocess_image na to aby sme mohli konvertovať obrázky spätne a zobraziť ich.

In [0]:
def deprocess_image(img):
  mean = np.array([103.939, 116.779, 123.68])
  img += mean
  img = np.clip(img, 0, 255)
  return np.uint8(img[:,:,::-1])

img = preprocess_input(test_imgs[0])
img = deprocess_image(img)
plt.figure(figsize = (10,10))
plt.imshow(img[:,:,::-1])
plt.show()

Toto je kód k základnému DeepDreamu. Všimnite si, že iterujeme tak aby sa loss zvyšovala. Pri tejto implementácii je dôležité uveodmiť si, že vhodné nastavenie eta úzko súvisí s tým ako je nastavená loss. Ak by loss bola napr. súčet a nie priemer, tak by sme zvolili nižšie eta.

In [0]:
def deep_dream(input_img, model, layer_name, channel_index, eta, steps):
  model_input = model.input
  layer_dict = dict([(layer.name, layer) for layer in model.layers])
  if layer_name not in layer_dict:
    raise ValueError('Layer ' + layer_name + ' not found in model.')
  layer = layer_dict[layer_name].output
  if layer.shape[3] < channel_index:
    raise ValueError('Layer ' + layer_name + ' has only {} channels.'.format(layer.shape[3]))
  
  loss = K.mean(K.square(layer[:, :, :, channel_index]))  
  grad = K.gradients(loss, model_input)[0]
  
  fetch_loss_grad = K.function([model_input], [loss, grad])

  for i in range(steps):
    out = fetch_loss_grad([input_img])
    r_loss = out[0]    
    print("At step: {}, loss: {}".format(i, r_loss))

    r_grad = out[1]
    input_img += eta * r_grad

  return input_img

Našu funkciu si teraz môžeme otestovať. Najskôr si pozrieme názvy vrstie.

In [0]:
print([(l.name) for l in resnet.layers[1:]])

Tento kód nám spustí DeepDream a zobrazí obrázok.

In [0]:
in_img = preprocess_input(test_imgs[0])[np.newaxis, :, :, :]
out_img = deep_dream(in_img, resnet, 'conv2_block1_1_conv', 10, 10000, 200)
decoded_img = deprocess_image(out_img[0])
plt.figure(figsize = (10,10))
plt.imshow(decoded_img[:,:,::-1])
plt.show()

In [0]:
in_img = preprocess_input(test_imgs[0])[np.newaxis, :, :, :]
out_img = deep_dream(in_img, resnet, 'conv5_block3_3_conv', 187, 20000, 200)
decoded_img = deprocess_image(out_img[0])
plt.figure(figsize = (10,10))
plt.imshow(decoded_img[:,:,::-1])
plt.show()

### Úloha - Oklamanie siete (Adversarial example)

Aby sme si demonštrovali limitácie modelu, tak si teraz ukážeme ako je možné relatívne málo upraviť obrázok a oklamať tak sieť. Toto spravíme tak, že aplikujeme podobný postup ako v DeepDream časti, avšak maximalizovať budeme hodnotu klasifikácie pre nejakú triedu a minimalizovať budeme aktiváciu pre správnu klasifikáciu.

Dopíšte funkcia adversarial dream, tak aby upravila obrázok aby ho klasifikovala ako inú triedu ako predtým.

Pozn.: V prípade, že sa vyrábajú tzv. adversarial examples sa často krát pridáva do loss function člen, ktorý zabezpečí, že zmena obrázka je čo najmenšia, to však nás až tak netrápi.

In [0]:
def adversarial_dream(input_img, model):
  return input_img

Ak je funkcia správne implementovaná tento kód by mal zobraziť upravený obrázok. A po volaní predict by mala sieť vrátiť inú triedu.

In [0]:
in_img = preprocess_input(test_imgs[0])[np.newaxis, :, :, :]
out_img = adversarial_dream(in_img, resnet)
prediction = resnet.predict(out_img)
print(np.max(prediction[0]))
print(decode_predictions(prediction))
decoded_img = deprocess_image(out_img[0])
plt.figure(figsize = (10,10))
plt.imshow(decoded_img[:,:,::-1])
plt.show()

Overíme si aj fungovanie po deprocesingu a processingu. Toto nemusí fungovať vždy úplne dobre, ale môžete sa s tým pohrať.

In [0]:
prediction = resnet.predict(preprocess_input(decoded_img)[np.newaxis, ...])
print(np.argmax(prediction[0]))
print(decode_predictions(prediction))

### Úloha 3

Teraz dopíšte túto funkciu tak, aby generovala obrázok, ktorý sieť klasifikuje ako nami zvolenú triedu podľa ID zo [zoznamu](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a). Myslite na to, že zvolená trieda môže mať pre počiatočný obrázok veľmi malú pravdepodobnosť. Toto musíte vo funckii zohľadniť. 

In [0]:
def adversarial_dream_with_target(input_img, model, desired_class_id):
  return input_img

In [0]:
in_img = preprocess_input(test_imgs[0])[np.newaxis, :, :, :]
out_img = adversarial_dream_with_target(in_img, resnet, 640)
prediction = resnet.predict(out_img)
print(np.argmax(prediction[0]))
print(decode_predictions(prediction))
decoded_img = deprocess_image(out_img[0])
plt.figure(figsize = (10,10))
plt.imshow(decoded_img[:,:,::-1])
plt.show()

Opäť sa uistíme, či nám zaokrúhlovanie a cliping hodnôt neuškodí v zámere.

In [0]:
prediction = resnet.predict(preprocess_input(decoded_img)[np.newaxis, ...])
print(np.argmax(prediction[0]))
print(decode_predictions(prediction))

Pre zaujímavosť si môžeme zobraziť aj rozdieľ obrázkov.

In [0]:
diff_img = decoded_img.astype(np.float32) - test_imgs[0].astype(np.float32)


print(np.max(diff_img))
print(np.min(diff_img))

diff_img /= np.max(np.abs(diff_img))

print(diff_img.dtype)

plt.figure(figsize = (10,10))
plt.imshow(decoded_img[:,:,::-1])
plt.show()
plt.figure(figsize = (10,10))
plt.imshow(test_imgs[0][:,:,::-1])
plt.show()
plt.figure(figsize = (10,10))
plt.imshow(diff_img[:,:,::-1])
plt.show()
plt.figure(figsize = (10,10))
plt.imshow(-diff_img[:,:,::-1])
plt.show()